In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.ticker as tck
import pickle
import math as maths
from scipy import interpolate
import matplotlib as mpl
from matplotlib import animation, rc
import time

from scipy.optimize import curve_fit
pi = np.pi

In [ ]:
#initial condition generator
def ic_generator(theta_lower_bound, theta_upper_bound, p_lower_bound, p_upper_bound, N): 
    #For 
    if not (theta_upper_bound == theta_lower_bound or p_upper_bound == p_lower_bound):
        theta_list = np.linspace(theta_lower_bound, theta_upper_bound, N)
        p_list = np.linspace(p_lower_bound, p_upper_bound, N)
    else:
        N = N**2
        if theta_upper_bound == theta_lower_bound:
            theta_list = [0]
            p_list = np.linspace(p_lower_bound, p_upper_bound, N)
        else:
            theta_list = np.linspace(theta_lower_bound, theta_upper_bound, N)
            p_list = [0]
    output_list = []
    for theta in theta_list:
        for p in p_list:
            output_list.append([theta, p])
    return output_list

#This is the function which simulates the system
def kicker(kick_num, k, initial_conditions):
    #The intitial conditions are in multiples of pi
    pi = np.pi
    kick_num = round(kick_num*1000)
    #Making the arrays
    theta = np.zeros((1, kick_num))[0]
    momenta = np.zeros((1, kick_num))[0]
    #Setting the initial_conditions
    theta[0] = initial_conditions[0]*pi
    momenta[0] = initial_conditions[1]*pi
    description = [kick_num, k, initial_conditions]
    for n in range(kick_num-1):
        #Mapping
        momenta[n+1] = momenta[n]+k*round(np.sin(theta[n]), 10)
        theta[n+1] = theta[n] + momenta[n+1]          
    return np.array([np.mod(theta,2.*pi), np.mod(momenta,2.*pi), description])


def chaining_multiple_data_sets_to_one(input_data):
    output_data = []
    for k_dataset in input_data:
        output_k_dataset = [[], [], []]
        for ic_datum in k_dataset:
            output_k_dataset[0] = list(it.chain(output_k_dataset[0], ic_datum[0]))
            output_k_dataset[1] = list(it.chain(output_k_dataset[1], ic_datum[1]))
        output_k_dataset[2] = k_dataset[0][2][1]
        output_data.append(output_k_dataset)
    return output_data

def fractal_dim_prepper(input_data):
    # each element of input_data is a simulation for a certain value of k
    data_length = len(input_data[0][0])
    #This is to choose the number of bin (or pixels) to use in each of the histograms. Sqrt because the  
    #bin number is the sqrt of the total number of bins. There is a log, because soon it will be exponentiated
    linear_list = np.arange(1, maths.ceil(np.log(maths.sqrt(data_length))), 0.5)
    #It is exponentiated so it will appear linear in a log plot
    exp_list = [np.e**element for element in linear_list]
    #Rounded as the number of bins needs to be integer
    rounded_exp_list = [int(round(x)) for x in exp_list]
    #Taking the value of k for each of the simulations
    k_list_func = [datum[2] for datum in input_data]
    nonzero_bins_list = []
    bin_num_list = rounded_exp_list
    line_fit_data_output = []
    l_list_func = []
    #This calculates the side-length of each bin/pixel for each of the bin_nums
    for bin_num in bin_num_list:
        l = 2*pi/bin_num
        l_list_func.append(l)
    #    
    scaling_list = [1/l for l in l_list_func]
    for input_datum in input_data:
        temp_N_list = []
        for bin_num in bin_num_list:
            #Making a histogram
            nphisto, xedges, yedges = np.histogram2d(input_datum[0], input_datum[1], bins = bin_num)
            #Counting the number of non-empty pixels and appending that to temp_N_list
            temp_N_list.append(np.count_nonzero(nphisto))
        #Fitting a line to the log-log plot of the scaling (x axis), and number of non-empty pixels (y-axis)
        popt, pcov = curve_fit(f, np.log(scaling_list), np.log(temp_N_list))
        nonzero_bins_list.append(temp_N_list)
        line_fit_stuff = [popt, pcov]
        line_fit_data_output.append(line_fit_stuff)
        #line_fit_data_output is a list of elements of [[slope, y_intercept], pcov] for each of the graphs
    return [scaling_list, nonzero_bins_list, k_list_func, line_fit_data_output]

#Line for fitting
def f(x, A, B): 
    return A*x + B


def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))


In [ ]:
kicker --> chaining_multiple_datasets_to_one --> fractal_dim_prepper --> Results --> Profit